In [2]:
import os
import pandas as pd
import pathlib

# pr dataset

In [5]:
list_color = ['RGB', 'grayscale']
list_image_size = ['256', '400', '512']
list_minimum_image = ['5', '10', '20']
path_dataset = '/home/xandao/Documentos/dataset_gimp/imagens_george/imagens/'

for color in list_color:
    for image_size in list_image_size:
        for minimum_image in list_minimum_image:
            p = os.path.join(path_dataset, color, 'specific_epithet', image_size, minimum_image)
            file_labels = [f for f in pathlib.Path(p).rglob('label2.txt') if f.is_file()]
            if len(file_labels) == 1:
                df = pd.read_csv(file_labels[0], header=None, sep=';', names=['species', 'folder', 'count'])
                display('color: %s image_size: %s minimum_image: %s' % (
                color, image_size, minimum_image))
                for folder_name in df['folder']:
                    df['list_images'] = str(
                        [str(image.stem) for image in pathlib.Path(os.path.join(p, folder_name)).rglob('*.jpeg') if
                         image.is_file()])
                display(df.head(5))
                filename = '%s_%s_%s.csv' % (color, image_size, minimum_image)
                filename = os.path.join('csv', 'pr', filename)
                df.to_csv(filename, sep=';', header=None, index=None, lineterminator='\n')

'color: RGB image_size: 256 minimum_image: 5'

,species,folder,count,list_images
0,corcovadensis,f1,68,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
1,regnellii,f2,17,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
2,mikanianum,f3,37,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
3,subretusa,f4,5,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
4,reitzii,f5,12,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."


'color: RGB image_size: 256 minimum_image: 10'

,species,folder,count,list_images
0,corcovadensis,f1,68,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
1,regnellii,f2,17,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
2,mikanianum,f3,37,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
3,reitzii,f4,12,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
4,miquelianum,f5,70,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."


'color: RGB image_size: 256 minimum_image: 20'

...

,species,folder,count,list_images
0,corcovadensis,f1,68,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
1,regnellii,f2,17,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
2,mikanianum,f3,37,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
3,subretusa,f4,5,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."
4,reitzii,f5,12,"['MBM037883', 'NY00559096_01', 'MBM045085', 'S..."


'color: grayscale image_size: 512 minimum_image: 10'

,species,folder,count,list_images
0,corcovadensis,f1,68,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
1,regnellii,f2,17,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
2,mikanianum,f3,37,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
3,reitzii,f4,12,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."
4,miquelianum,f5,70,"['MBM089701', 'MBM089707', 'NY00559303_01', 'M..."


'color: grayscale image_size: 512 minimum_image: 20'

,species,folder,count,list_images
0,corcovadensis,f1,68,"['ESA100586', 'NY00555607_01', 'MBM171931', 'H..."
1,mikanianum,f2,37,"['ESA100586', 'NY00555607_01', 'MBM171931', 'H..."
2,miquelianum,f3,70,"['ESA100586', 'NY00555607_01', 'MBM171931', 'H..."
3,crassinervium,f4,20,"['ESA100586', 'NY00555607_01', 'MBM171931', 'H..."
4,gaudichaudianum,f5,105,"['ESA100586', 'NY00555607_01', 'MBM171931', 'H..."
